## 5. Single Point of Input

We now design the script such that uses an intake catalog with only kerchunked datasets as input.

In [19]:
%%writefile xpublish_spoi.py

port=9000
ssl_keyfile="/work/bm0021/k204210/cloudify/workshop/key.pem"
ssl_certfile="/work/bm0021/k204210/cloudify/workshop/cert.pem"

from cloudify.plugins.stacer import *
from cloudify.utils.daskhelper import *
from cloudify.plugins.kerchunk import *
import xarray as xr
import xpublish as xp
import asyncio
import nest_asyncio
import sys
import os
import intake

nest_asyncio.apply()
STORAGE_OPTIONS=dict(
    cache_size=0
)

def get_options(desc):
    options = dict(storage_options=desc["args"].get("storage_options", {}))
    options["storage_options"].update(STORAGE_OPTIONS)
    return options

if __name__ == "__main__":  # This avoids infinite subprocess creation
    import dask
    zarrcluster = asyncio.get_event_loop().run_until_complete(get_dask_cluster())
    os.environ["ZARR_ADDRESS"]=zarrcluster.scheduler._address
    
    caturi=sys.argv[1]
    cat = intake.open_catalog(caturi)

    dsdict={}
    mapper_dict={}
    for dsname in list(cat):
        desc = cat[dsname].describe()
        testurl = desc["args"]["urlpath"]
        if type(testurl)!=str:
            continue
        elif not testurl.startswith("reference::"):
            continue
        options=get_options(desc)
        ds=cat[dsname](**options).to_dask()

        sp = ds.encoding["source"]
        ds=ds.drop_encoding()   
        ds.encoding["source"] = sp
        
        mapper_dict[sp] = fsspec.get_mapper(sp, **options)
        dsdict[dsname]=ds
        
    kp = KerchunkPass()
    kp.mapper_dict = mapper_dict

    collection = xp.Rest(dsdict)
    collection.register_plugin(Stac())
    collection.register_plugin(kp)
    collection.serve(
        host="0.0.0.0",
        port=port,
        ssl_keyfile=ssl_keyfile,
        ssl_certfile=ssl_certfile
    )

Writing xpublish_spoi.py


We run this app with an intake catalog for all [kerchunked ERA5 data](://gitlab.dkrz.de/data-infrastructure-services/era5-kerchunks/-/raw/main/main.yaml).

In [20]:
%%bash --bg
source activate /work/bm0021/conda-envs/cloudify
python xpublish_spoi.py https://gitlab.dkrz.de/data-infrastructure-services/era5-kerchunks/-/raw/main/main.yaml

If sth goes wrong, you can check for *cloudify* processes that you can *kill* by ID.

In [30]:
!ps -ef | grep cloudify

k204210  1101263 1100112  0 07:33 ?        00:00:02 /work/bm0021/conda-envs/cloudify/bin/python -Xfrozen_modules=off -m ipykernel_launcher -f /home/k/k204210/.local/share/jupyter/runtime/kernel-edc24184-27e4-4966-beb3-6f926e2d6cf0.json
k204210  1101280 1100112  0 07:33 ?        00:00:02 /work/bm0021/conda-envs/cloudify/bin/python -Xfrozen_modules=off -m ipykernel_launcher -f /home/k/k204210/.local/share/jupyter/runtime/kernel-99eb3193-dfc0-45c3-9953-a1f29fb0888d.json
k204210  1101285 1100112  0 07:33 ?        00:00:02 /work/bm0021/conda-envs/cloudify/bin/python -Xfrozen_modules=off -m ipykernel_launcher -f /home/k/k204210/.local/share/jupyter/runtime/kernel-0ac5cfbe-7cc6-4136-870a-cbac9aa18a7b.json
k204210  1101286 1100112  0 07:33 ?        00:00:02 /work/bm0021/conda-envs/cloudify/bin/python -Xfrozen_modules=off -m ipykernel_launcher -f /home/k/k204210/.local/share/jupyter/runtime/kernel-947ea9d3-56f0-426e-9af8-8a5be0180793.json
k204210  1102668 1100112  0 09:10 ?        00:00:21 /wor

In [31]:
!kill 1104568

In [23]:
port=9000
hostname=!echo $HOSTNAME
hosturl="https://"+hostname[0]+":"+str(port)
intake_url='/'.join([hosturl,"intake.yaml"])
print(intake_url)

https://l40072.lvt.dkrz.de:9000/intake.yaml


In [24]:
import intake
storage_options=dict(verify_ssl=False)
cat=intake.open_catalog(
    intake_url,
    storage_options=storage_options
)
list(cat)

['pressure-level_analysis_daily',
 'pressure-level_analysis_monthly',
 'surface_analysis_daily',
 'surface_analysis_hourly',
 'surface_analysis_monthly',
 'surface_forecast_hourly',
 'surface_forecast_monthly']

In [26]:
dsname='pressure-level_analysis_monthly'
ds=cat[dsname](storage_options=storage_options).to_dask()
ds

/work/bm0021/conda-envs/cloudify/lib/python3.11/site-packages/intake_xarray/base.py:21: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  'dims': dict(self._ds.dims),


<xarray.Dataset> Size: 3TB
Dimensions:  (time: 1008, level: 37, value: 542080)
Coordinates:
    lat      (value) float64 4MB dask.array<chunksize=(542080,), meta=np.ndarray>
  * level    (level) int64 296B 1 2 3 5 7 10 20 ... 850 875 900 925 950 975 1000
    lon      (value) float64 4MB dask.array<chunksize=(542080,), meta=np.ndarray>
  * time     (time) datetime64[ns] 8kB 1940-01-01T12:00:00 ... 2023-12-01T12:...
Dimensions without coordinates: value
Data variables: (12/16)
    cc       (time, level, value) float64 162GB dask.array<chunksize=(1, 1, 542080), meta=np.ndarray>
    ciwc     (time, level, value) float64 162GB dask.array<chunksize=(1, 1, 542080), meta=np.ndarray>
    clwc     (time, level, value) float64 162GB dask.array<chunksize=(1, 1, 542080), meta=np.ndarray>
    crwc     (time, level, value) float64 162GB dask.array<chunksize=(1, 1, 542080), meta=np.ndarray>
    cswc     (time, level, value) float64 162GB dask.array<chunksize=(1, 1, 542080), meta=np.ndarray>
    d        (time, level, value) float64 162GB dask.array<chunksize=(1, 1, 542080), meta=np.ndarray>
    ...       ...
    t        (time, level, value) float64 162GB dask.array<chunksize=(1, 1, 542080), meta=np.ndarray>
    u        (time, level, value) float64 162GB dask.array<chunksize=(1, 1, 542080), meta=np.ndarray>
    v        (time, level, value) float64 162GB dask.array<chunksize=(1, 1, 542080), meta=np.ndarray>
    vo       (time, level, value) float64 162GB dask.array<chunksize=(1, 1, 542080), meta=np.ndarray>
    w        (time, level, value) float64 162GB dask.array<chunksize=(1, 1, 542080), meta=np.ndarray>
    z        (time, level, value) float64 162GB dask.array<chunksize=(1, 1, 542080), meta=np.ndarray>
Attributes: (12/22)
    project:              ECMWF Re-Analysis
    project_id:           ERA
    institution_id:       ECMWF-DKRZ
    institution:          Data from European Centre for Medium-Range Weather ...
    source_id:            IFS
    source:               ECMWF Integrated Forecast System (IFS) CY41R2
    ...                   ...
    format:               kerchunk
    product:              reanalysis
    responsible_persons:  Angelika Heil, Fabian Wachsmann
    title:                The DKRZ ERA5 data pool. Generated using Copernicus...
    license:              The ERA5 data are published with the Copernicus Pro...
    references:           Hersbach, H., Bell, B., Berrisford, P., Hirahara, S...

In [29]:
ds["t"].isel(time=434,level=35).load()

<xarray.DataArray 't' (value: 542080)> Size: 4MB
array([251.69160461, 251.71504211, 251.71113586, ..., 244.89863586,
       245.12324524, 245.40840149])
Coordinates:
    lat      (value) float64 4MB 89.78 89.78 89.78 ... -89.78 -89.78 -89.78
    level    int64 8B 975
    lon      (value) float64 4MB 0.0 20.0 40.0 60.0 ... 280.0 300.0 320.0 340.0
    time     datetime64[ns] 8B 1976-03-01T12:00:00
Dimensions without coordinates: value
Attributes: (12/14)
    paramId:                    130
    numberOfPoints:             542080
    typeOfLevel:                isobaricInhPa
    stepUnits:                  1
    stepType:                   avg
    gridType:                   reduced_gg
    ...                         ...
    name:                       Temperature
    cfName:                     air_temperature
    cfVarName:                  t
    missingValue:               9999
    NV:                         0
    gridDefinitionDescription:  Gaussian Latitude/Longitude Grid